In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/messy-imdb-dataset/messy_IMDB_dataset.csv


In [2]:
df = pd.read_csv('/kaggle/input/messy-imdb-dataset/messy_IMDB_dataset.csv', encoding = "ISO-8859-1", delimiter=";")

In [3]:
df.head()

,IMBD title ID,Original titlÊ,Release year,Genrë¨,Duration,Country,Content Rating,Director,Unnamed: 8,Income,Votes,Score
0,tt0111161,The Shawshank Redemption,1995-02-10,Drama,142,USA,R,Frank Darabont,NaN,$ 28815245,2.278.845,9.3
1,tt0068646,The Godfather,09 21 1972,"Crime, Drama",175,USA,R,Francis Ford Coppola,NaN,$ 246120974,1.572.674,9.2
2,tt0468569,The Dark Knight,23 -07-2008,"Action, Crime, Drama",152,US,PG-13,Christopher Nolan,NaN,$ 1005455211,2.241.615,9.
3,tt0071562,The Godfather: Part II,1975-09-25,"Crime, Drama",220,USA,R,Francis Ford Coppola,NaN,"$ 4o8,035,783",1.098.714,"9,.0"
4,tt0110912,Pulp Fiction,1994-10-28,"Crime, Drama",,USA,R,Quentin Tarantino,NaN,$ 222831817,1.780.147,"8,9f"


## TO-DO
1. The column names needs to be fixed.
2. Release year column has inconsistent format of dates
3. In country column, countries like USA has synonyms like US and others are wrongly spelled.
4. Undefined columns.
5. Removal of special symbols from Income,Votes and Score columns.

In [4]:
df.shape

(101, 12)

In [5]:
df.columns

Index(['IMBD title ID', 'Original titlÊ', 'Release year', 'Genrë¨', 'Duration',
       'Country', 'Content Rating', 'Director', 'Unnamed: 8', 'Income',
       ' Votes ', 'Score'],
      dtype='object')

In [6]:
#Renaming columns to represent proper attribute names
columns_mapping = {df.columns[0]:"IMDB ID",df.columns[1]:"original title",df.columns[3]:"genre",df.columns[8]:"unknown",df.columns[10]:df.columns[10].strip()}
renamedf = df.rename(columns=columns_mapping)

In [7]:
renamedf.columns

Index(['IMDB ID', 'original title', 'Release year', 'genre', 'Duration',
       'Country', 'Content Rating', 'Director', 'unknown', 'Income', 'Votes',
       'Score'],
      dtype='object')

In [8]:
renamedf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   IMDB ID         100 non-null    object 
 1   original title  100 non-null    object 
 2   Release year    100 non-null    object 
 3   genre           100 non-null    object 
 4   Duration        99 non-null     object 
 5   Country         100 non-null    object 
 6   Content Rating  77 non-null     object 
 7   Director        100 non-null    object 
 8   unknown         0 non-null      float64
 9   Income          100 non-null    object 
 10  Votes           100 non-null    object 
 11  Score           100 non-null    object 
dtypes: float64(1), object(11)
memory usage: 9.6+ KB


All the 11 attributes has atleast one Null value in them.

In [9]:
#Dropping the 'unknown' empty column and the Imdb ID column
renamedf.drop('unknown',axis=1,inplace=True)
renamedf.drop('IMDB ID',axis=1,inplace=True)

In [10]:
#Analzing the ReleaseYear column values
renamedf.iloc[:,1].unique()

array(['1995-02-10', '09 21 1972', ' 23 -07-2008', '1975-09-25',
       '1994-10-28', '22 Feb 04', '1994-03-11', '1957-09-04',
       '2010-09-24', '10-29-99', '1994-10-06', '2002-01-18',
       '23rd December of 1966 ', nan, '1999-05-07', '01/16-03',
       '1980-09-19', '1990-09-20', '18/11/1976', '2014-11-06',
       '1995-12-15', '1991-03-05', '1977-10-20', '1998-10-30',
       '2000-10-03', '2003-05-09', '2003-04-18', '1997-12-20',
       '2019-11-07', '1948-03-11', '1955-08-19', '2000-05-19',
       '2006-10-27', '2006-12-22', '1985-10-18', '1999-08-27',
       '1995-04-07', '1991-12-19', '1995-11-30', '1994-11-25',
       '2019-10-03', '2012-02-24', '2002-10-25', '2015-02-12',
       '1960-10-28', '21-11-46', '1968-12-21', '2015-10-11', '1988-11-17',
       '1937-03-12', '2012-08-29', '2013-01-17', '2001-01-19',
       '2008-10-17', '1980-12-22', '1981-06-12', '2018-04-25',
       '1979-10-25', '2019-04-24', '1979-12-18', '2005-05-06',
       '1964-04-03', '1955-04-14', '2017-12

The release year attribute has varying formats of dates, we need to convert it to a common format(ex. yyyy-mm-dd)

In [11]:
import re
import datetime
from datetime import datetime

#Removes noise from date values and convert to standard datetime strings
def formatDates(messyDate):
    dateStr = messyDate.strip()
    dateStr = re.sub(r'(\d)(st|nd|rd|th)', r'\1', dateStr)
    dateStr = re.sub(' [a-zA-Z]{2} '," ",dateStr)
    pattern = '( -|/)|(-|/ )|( -|/ )'
   
    if re.search(pattern,dateStr):

        dateStr=re.sub(pattern=pattern,string=dateStr,repl="-")
    
    
    dateParsers = ('%Y-%m-%d','%d-%m-%Y','%d-%m-%y','%d %b %Y','%d %b %y','%d %B %Y','%m-%d-%y','%m %d %Y')
    isValidDate = False
    for parser in dateParsers:
        try:
            dateObj = datetime.strptime(dateStr,parser)
#             dateStr = str(dateObj)
            dateStr = dateObj.date()   
            isValidDate = True
        except Exception as e:
            pass
            
    if isValidDate:
        return  str(dateStr)
    else:
        #Dates which dosent lie in the above mentioned date formats(dateParsers) will not be considered
        return None
        
    

In [12]:
renamedf.iloc[:,1] = renamedf.iloc[:,1].map(formatDates,na_action='ignore')


In [13]:
renamedf.iloc[:,1] = renamedf.iloc[:,1].str.replace('None',"")

In [14]:
renamedf.iloc[:,1].unique()

array(['1995-02-10', '1972-09-21', '2008-07-23', '1975-09-25',
       '1994-10-28', '2004-02-22', '1994-03-11', '1957-09-04',
       '2010-09-24', '1999-10-29', '1994-10-06', '2002-01-18',
       '1966-12-23', nan, '1999-05-07', '2003-01-16', '1980-09-19',
       '1990-09-20', '1976-11-18', '2014-11-06', '1995-12-15',
       '1991-03-05', '1977-10-20', '1998-10-30', '2000-10-03',
       '2003-05-09', '2003-04-18', '1997-12-20', '2019-11-07',
       '1948-03-11', '1955-08-19', '2000-05-19', '2006-10-27',
       '2006-12-22', '1985-10-18', '1999-08-27', '1995-04-07',
       '1991-12-19', '1995-11-30', '1994-11-25', '2019-10-03',
       '2012-02-24', '2002-10-25', '2015-02-12', '1960-10-28',
       '2046-11-21', '1968-12-21', '2015-10-11', '1988-11-17',
       '1937-03-12', '2012-08-29', '2013-01-17', '2001-01-19',
       '2008-10-17', '1980-12-22', '1981-06-12', '2018-04-25',
       '1979-10-25', '2019-04-24', '1979-12-18', '2005-05-06',
       '1964-04-03', '1955-04-14', '2017-12-28', '

In [15]:
#Analyzing genre column
renamedf['genre'].unique()

array(['Drama', 'Crime, Drama', 'Action, Crime, Drama',
       'Action, Adventure, Drama', 'Biography, Drama, History',
       'Action, Adventure, Sci-Fi', 'Drama, Romance', 'Western', nan,
       'Action, Sci-Fi', 'Action, Adventure, Fantasy',
       'Biography, Crime, Drama', 'Adventure, Drama, Sci-Fi',
       'Crime, Drama, Mystery', 'Crime, Drama, Thriller', 'Drama, War',
       'Crime, Drama, Fantasy', 'Animation, Adventure, Family',
       'Comedy, Drama, Romance', 'Comedy, Drama, Thriller',
       'Drama, Family, Fantasy', 'Drama, Mystery, Sci-Fi',
       'Adventure, Comedy, Sci-Fi', 'Crime, Mystery, Thriller',
       'Animation, Adventure, Drama', 'Biography, Comedy, Drama',
       'Biography, Drama, Music', 'Drama, Music',
       'Horror, Mystery, Thriller', 'Drama, Romance, War',
       'Animation, Drama, War', 'Comedy, Drama, Family',
       'Action, Adventure', 'Drama, Western', 'Mystery, Thriller',
       'Drama, Horror', 'Horror, Sci-Fi', 'Drama, Mystery, War',
       'Ac

Genre attribute can have multiple genre for each entry, so we'll convert the multivalue genre column into multiple columns having unique genre 

In [16]:
import re
#cleaning the genre values
renamedf.genre = renamedf.genre.apply(lambda multigenre:re.sub(pattern=' ',repl="",string=str(multigenre)))

In [17]:
#One hot encoding the features
genredf=renamedf.genre.str.get_dummies(',')

In [18]:

mergeddf = pd.concat([renamedf,genredf],axis=1)
mergeddf.drop('genre',inplace=True,axis=1)

In [19]:
mergeddf.head()

,original title,Release year,Duration,Country,Content Rating,Director,Income,Votes,Score,Action,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,nan
0,The Shawshank Redemption,1995-02-10,142,USA,R,Frank Darabont,$ 28815245,2.278.845,9.3,0,...,0,0,0,0,0,0,0,0,0,0
1,The Godfather,1972-09-21,175,USA,R,Francis Ford Coppola,$ 246120974,1.572.674,9.2,0,...,0,0,0,0,0,0,0,0,0,0
2,The Dark Knight,2008-07-23,152,US,PG-13,Christopher Nolan,$ 1005455211,2.241.615,9.,1,...,0,0,0,0,0,0,0,0,0,0
3,The Godfather: Part II,1975-09-25,220,USA,R,Francis Ford Coppola,"$ 4o8,035,783",1.098.714,"9,.0",0,...,0,0,0,0,0,0,0,0,0,0
4,Pulp Fiction,1994-10-28,,USA,R,Quentin Tarantino,$ 222831817,1.780.147,"8,9f",0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
mergeddf.Country.unique()

array(['USA', 'US', 'New Zealand', 'UK', 'New Zesland', 'Italy', nan,
       'New Zeland', 'US.', 'Brazil', 'Japan', 'Italy1', 'South Korea',
       'France', 'Germany', 'India', 'Denmark', 'West Germany', 'Iran'],
      dtype=object)

Country names cant have digits and special characters in them.Hence we'll remove it

In [21]:
mergeddf.Country = mergeddf.Country.map(lambda x:re.sub('[^a-z-A-Z\s]',"",x),na_action='ignore')

Some countries have have slight inaccuracies in their names (For ex. USA,US etc.)

In [22]:
from difflib import SequenceMatcher

#let's specify correct names
correct_names = {'USA','New Zealand','UK','Italy','South Korea','France','Iran','Germany','Denmark','India','Brazil','Japan'
                }

#This function will find the best match for a country name having errors in their names
def get_most_similar(word,wordlist):
    top_similarity = 0.0
    most_similar_word = word
    for candidate in wordlist:
        similarity = SequenceMatcher(None,word,candidate).ratio()
        if similarity > top_similarity:
                
            top_similarity = similarity
            most_similar_word = candidate
    return most_similar_word

#Applying this function over 'country' column in dataframe
mergeddf.Country = mergeddf.Country.map(lambda x: get_most_similar(x,correct_names),na_action='ignore')

In [23]:
mergeddf.Country.value_counts()

USA            64
UK             12
Italy           5
Japan           4
New Zealand     3
France          3
Germany         3
South Korea     2
Brazil          1
India           1
Denmark         1
Iran            1
Name: Country, dtype: int64

In [24]:
mergeddf.Duration.unique()

array(['142', '175', '152', '220', ' ', '201', 'Nan', '96', '148', 'Inf',
       '178c', '161', nan, '179', 'Not Applicable', '146', '-', '169',
       '127', '118', '121', '189', '130', '125', '116', '132', '207',
       '155', '151', '119', '110', '137', '106', '88', '122', '112',
       '150', '109', '102', '165', '89', '87', '164', '113', '98', '115',
       '149', '117', '181', '147', '120', '95', '105', '170', '134',
       '229', '153', '178', '131', '99', '108', '81', '126', '104', '136',
       '103', '114', '160', '128', '228', '129', '123'], dtype=object)

In [25]:
#Cleaning invalid duration values
mergeddf.Duration =mergeddf.Duration.map(lambda x:re.sub('[^0-9]',"",x),na_action='ignore')

In [26]:
mergeddf['Content Rating'].value_counts()

R            45
PG-13        12
PG           11
G             6
Not Rated     1
Approved      1
Unrated       1
Name: Content Rating, dtype: int64

In [27]:
#The content ratings 'Not rated' and 'unrated' means the same
mergeddf['Content Rating'].replace(to_replace='Not Rated',value='Unrated',inplace=True)

In [28]:
mergeddf['Director'].value_counts()

Christopher Nolan                  6
Stanley Kubrick                    5
Alfred Hitchcock                   4
Quentin Tarantino                  4
Sergio Leone                       4
                                  ..
Isao Takahata                      1
Giuseppe Tornatore                 1
Lana Wachowski, Lilly Wachowski    1
Andrew Stanton                     1
Stanley Donen                      1
Name: Director, Length: 64, dtype: int64

In [29]:
#Director attribute can have more than 1 director entry per record, so one hot encoding it
directorsDf = mergeddf['Director'].str.get_dummies()
pd.concat([mergeddf,directorsDf],axis=1)

,original title,Release year,Duration,Country,Content Rating,Director,Income,Votes,Score,Action,...,Sam Mendes,Sergio Leone,Sidney Lumet,Stanley Donen,Stanley Kubrick,Steven Spielberg,Thomas Vinterberg,Todd Phillips,Tony Kaye,Wolfgang Petersen
0,The Shawshank Redemption,1995-02-10,142,USA,R,Frank Darabont,$ 28815245,2.278.845,9.3,0,...,0,0,0,0,0,0,0,0,0,0
1,The Godfather,1972-09-21,175,USA,R,Francis Ford Coppola,$ 246120974,1.572.674,9.2,0,...,0,0,0,0,0,0,0,0,0,0
2,The Dark Knight,2008-07-23,152,USA,PG-13,Christopher Nolan,$ 1005455211,2.241.615,9.,1,...,0,0,0,0,0,0,0,0,0,0
3,The Godfather: Part II,1975-09-25,220,USA,R,Francis Ford Coppola,"$ 4o8,035,783",1.098.714,"9,.0",0,...,0,0,0,0,0,0,0,0,0,0
4,Pulp Fiction,1994-10-28,,USA,R,Quentin Tarantino,$ 222831817,1.780.147,"8,9f",0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,The Sting,1974-03-21,129,USA,PG,George Roy Hill,$ 156000000,236.285,7.5,0,...,0,0,0,0,0,0,0,0,0,0
97,Das Boot,1982-03-18,149,Germany,R,Wolfgang Petersen,$ 11487676,226.427,7.5,0,...,0,0,0,0,0,0,0,0,0,1
98,Per qualche dollaro in piÃ¹,1965-12-20,132,Italy,NaN,Sergio Leone,$ 15000000,226.039,7.4,0,...,0,1,0,0,0,0,0,0,0,0
99,Jodaeiye Nader az Simin,2011-10-21,123,Iran,PG-13,Asghar Farhadi,$ 22926076,214.165,7.4,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
mergeddf.Votes.unique()

array(['2.278.845', '1.572.674', '2.241.615', '1.098.714', '1.780.147',
       '1.604.280', '1.183.248', '668.473', '2.002.816', '1.807.440',
       '1.755.490', '1.619.920', '672.499', nan, '1.632.315', '1.449.778',
       '1.132.073', '991.505', '891.071', '1.449.256', '1.402.015',
       '1.234.134', '1.204.107', '1.203.825', '1.112.336', '685.856',
       '626.693', '605.648', '470.931', '388.310', '307.958', '1.308.193',
       '1.159.703', '1.155.723', '1.027.330', '1.014.218', '1.007.598',
       '974.970', '968.947', '917.248', '855.097', '736.691', '707.942',
       '690.732', '586.765', '509.953', '295.220', '225.438', '223.050',
       '211.250', '1.480.582', '1.317.856', '1.098.879', '974.734',
       '869.480', '865.510', '796.486', '768.874', '754.786', '591.251',
       '501.082', '441.115', '432.390', '352.455', '349.642', '335.892',
       '332.217', '331.045', '302.317', '197.381', '195.789', '1.229.958',
       '1.049.009', '941.683', '928.036', '896.551', '889.875',

The votes are expressed in notation of million.thousands.hundreds, we need to remove the seperators in between to get numerical counts

In [31]:
mergeddf.Votes=mergeddf.Votes.str.replace('.',"")


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [32]:
mergeddf.Votes.isnull().sum()

1

In [33]:
mergeddf.Votes.fillna(0,inplace=True)

In [34]:
mergeddf.Votes.astype('int')

0      2278845
1      1572674
2      2241615
3      1098714
4      1780147
        ...   
96      236285
97      226427
98      226039
99      214165
100     213152
Name: Votes, Length: 101, dtype: int64

In [35]:
mergeddf.Income.unique()

array(['$ 28815245', '$ 246120974', '$ 1005455211', '$ 4o8,035,783',
       '$ 222831817', '$ 1142271098', '$ 322287794', '$ 576',
       '$ 869784991', '$ 101218804', '$ 678229452', '$ 887934303',
       '$ 25252481', nan, '$ 465718588', '$ 951227416', '$ 549265501',
       '$ 46879633', '$ 108997629', '$ 696742056', '$ 327333559',
       '$ 272753884', '$ 775768912', '$ 482349603', '$ 286801374',
       '$ 30680793', '$ 355467056', '$ 230098753', '$ 257604912',
       '$ 6130720', '$ 322773', '$ 465361176', '$ 291465034',
       '$ 109676311', '$ 388774684', '$ 23875127', '$ 19552639',
       '$ 520884847', '$ 23341568', '$ 968511805', '$ 1074251311',
       '$ 426588510', '$ 120072577', '$ 48983260', '$ 32008644',
       '$ 4374761', '$ 112911', '$ 516962', '$ 13826605', '$ 457688',
       '$ 1081133191', '$ 425368238', '$ 39970386', '$ 521311860',
       '$ 46520613', '$ 390133212', '$ 2048359754', '$ 108110316',
       '$ 2797800564', '$ 91968688', '$ 15002116', '$ 9443876',
     

In [36]:
#Removing special symbols from Income attribute and correcting some errors like 'o' instead of 0.
mergeddf.Income=mergeddf.Income.map(lambda income:re.sub('[^0-9o]',"",income),na_action='ignore')
mergeddf.Income = mergeddf.Income.str.replace('o','0')

In [37]:
mergeddf.Score.unique()

array(['9.3', '9.2', '9.', '9,.0', '8,9f', '08.9', '8.9', '8..8', '8.8',
       '8:8', nan, '++8.7', '8.7.', '8,7e-0', '8.7', '8.6', '8,6', '8.5',
       '8.4', '8.3', '8.2', '8.1', '8.0', '7.9', '7.8', '7.7', '7.6',
       '7.5', '7.4'], dtype=object)

In [38]:
mergeddf.Score = mergeddf.Score.map(lambda score:re.sub('[^\d]',"",score),na_action='ignore')

In [39]:
mergeddf.Score.unique()

array(['93', '92', '9', '90', '89', '089', '88', nan, '87', '870', '86',
       '85', '84', '83', '82', '81', '80', '79', '78', '77', '76', '75',
       '74'], dtype=object)

In [40]:
def cleanScores(score):
    score = str(score)
    if score.startswith('0'):
        score=score[1:]
       
    return score[0] + '.' + score[1:]

mergeddf.Score= mergeddf.Score.map(lambda score: cleanScores(score),na_action='ignore')

In [41]:
mergeddf.Score.unique()

array(['9.3', '9.2', '9.', '9.0', '8.9', '8.8', nan, '8.7', '8.70', '8.6',
       '8.5', '8.4', '8.3', '8.2', '8.1', '8.0', '7.9', '7.8', '7.7',
       '7.6', '7.5', '7.4'], dtype=object)

In [42]:
mergeddf.Score.fillna(0,inplace=True)

In [43]:
mergeddf.Score.astype('float')

0      9.3
1      9.2
2      9.0
3      9.0
4      8.9
      ... 
96     7.5
97     7.5
98     7.4
99     7.4
100    7.4
Name: Score, Length: 101, dtype: float64